# CVM. Выбор и проверка критерия для метрик конверсии

## Цели и описание ноутбука
Эксперименты CVM, целевыми метриками которых являются метрики конверсии, могут помочь при решении следующих задач:
- Сравнение конверсии группы с константным значением конверсии
- Сравнение больших конверсий между группами
- Сравнение малых конверсий между группами

Целями данного ноутбука являются:
1. Описание критериев, которые необходимо применять в каждой из вышеописанных задач
2. Описание функций, которые рассчитывают размер выборки и минимальный детектируемый эффект для каждого критерия
3. Описание функций, которые проверяют корректность выбора критерия на синтетических данных
4. Описание функций, которые строят доверительные интервалы для разницы между целевыми метриками
5. Опиание функций, которые используют для проверки статистической значимости при множественном тестировании

In [1]:
# Magic function to autoreload kernel when python module's code is changed
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')

## 1. Критерии для метрик конверсий

Критерий (или статистический тест) — это правило, по которому на основании выборочных данных принимается решение: принимать или отвергать нулевую гипотезу $H_0$. При этом критерий основывается на случайной величине $T(X_1, X_2, ... , X_n)$, называемой статистикой критерия.

Основные условия, которым должна удовлетворять случайная величина, чтобы считаться критерием:
1. Определена как функция от выборки
2. Известно (или выводимо) распределение при $H_0$
3. Чувствительность к отклонению от $H_0$ - при верной альтернативе $H_1$ распределение $T$ должно изменяться таким образом, чтобы вероятность попасть в критическую область возрастала
4. Монотонность / направленность - для односторонних тестов значение $T$ должно возрастать (или убывать) при переходе от $H_0$ к $H_1$, чтобы можно было задать однозначную «область отклонения» (чем больше отклонение, тем больше мощность теста)
5. Независимость от параметров, не проверяемых гипотезой (инвариантность)
6. Статистика критерия должна быть вычислимой

При решении задач сравнения конверсий необходимо применять следующие критерии:
1. Сравнение конверсии в группе с константным значением:
    - Биномиальный тест (Binomial test)
    - Одновыборочный T-тест (One-sample T-test)
2. Сравнение больших конверсий между группами:
    - Z-тест для пропорций (Z-test)
    - Двухвыборочный T-тест (Welch's T-test)
3. Сравнение малых конверсий между группами:
    - Точный тест Фишера (Fisher's Exact Test)
    - Бутстрап (Bootstrap) (применим ко всем трём задачам)

### 1.1.1. Сравнение конверсии в группе с константой. **Биномиальный тест**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: p = p_0 \\
H_1: p <> p_0
$$
где $p$ - конверсия в экспериментальной группе, $p_0$ - константа, с которой необходимо сравнить конверсию

#### Условия применимости критерия
1. Результаты экспериментов - независимые испытания Бернулли
2. Не требует большого размера выборки $n$, критерий работает при малых конверсиях $p$

#### Статистика критерия
Статистикой критерия является количество успехов $k$ в $n$ испытаниях. При верности $H_0$ статистика имеет биномиальное распределение с параметрами $n$ и $p_0$. $k ~ Binomial(n, p_0)$. 

#### Расчёт p-value
$p-value$ в данном случае есть сумма вероятностей всех исходов, вероятность которых меньше либо равна вероятности наблюдаемого $k$.

**Тренировочная задача №1**

**Условие:** представим, что историческая конверсия в целевое действие у сегмента равна $p_0 = 0.7$%. Мы отправляем выборке из 1000 клиентов СМС с призывом совершить целевое действие. После эксперимента мы выяснили, что в экспериментальной группе в целевое действие сконвертировалось 14 клиентов.

**Задача:** сравнить конверисю в экспериментальной группе ($p$) с исторической конверсией по сегменту ($p_0$) при помощи **Биномиального теста** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $p$ не равна $p_0$):
$$
H_0: p = 0.007 \\
H_1: p <> 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p$ больше $p_0$):
$$
H_0: p \le 0.007 \\
H_1: p > 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $p$ меньше $p_0$):
$$
H_0: p \ge 0.007 \\
H_1: p < 0.007 \\
$$

In [3]:
from ab_utils.binomial_test import binomial_p_value, binomial_p_value_manual

In [4]:
n = 1000          # количество пользователей (испытаний)
k = 14            # число конверсий (успехов)
p0 = 0.007        # ожидаемая конверсия 0.7% = 0.007
alpha = 0.05      # уровень значимости alpha

p = k / n
print(f"Наблюдаемая конверсия: {p:.1%}")
print(f"Ожидаемая конверсия (H0): {p0:.1%}")

Наблюдаемая конверсия: 1.4%
Ожидаемая конверсия (H0): 0.7%


In [5]:
alt_to_h_zero = {
    'two-sided': f'p {round(p * 100.0, 1)}% равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% больше либо равна p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% меньше либо равна p_0 {round(p0 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p {round(p * 100.0, 1)}% НЕ равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% меньше p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% больше p_0 {round(p0 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = binomial_p_value(k, n, p0, alternative = alt)
    p_manual = binomial_p_value_manual(k, n, p0, alternative = alt)

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (scipy):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")
    print(f"p-value (manual): {p_manual:.6f}")


Альтернатива: two-sided
p-value (scipy):  0.019676; Гипотеза H_0 (p 1.4% равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% НЕ равна p_0 0.7%): да
p-value (manual): 0.019676

Альтернатива: less
p-value (scipy):  0.994456; Гипотеза H_0 (p 1.4% больше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% меньше p_0 0.7%): нет
p-value (manual): 0.994456

Альтернатива: greater
p-value (scipy):  0.012514; Гипотеза H_0 (p 1.4% меньше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% больше p_0 0.7%): да
p-value (manual): 0.012514


### 1.1.2. Сравнение конверсии в группе с константой. **Одновыборочный T-тест**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: \mu = \mu_0 \\
H_1: \mu <> \mu_0
$$
где $\mu$ - среднее в экспериментальной группе, $\mu_0$ - константа, с которой необходимо сравнить среднее

#### Условия применимости критерия
1. Наблюдения независимы
2. Выборка из нормального распределения, либо $n$ достаточно велико ($n > 100$), чтобы (согласно ЦПТ) среднее было нормально распределено

#### Статистика критерия
Пусть
- $\bar{X}$ - выборочное среднее (среднее в экспериментальной группе),
- $s^2$ - несмещённая дисперсия,
- $n$ - размер выборки.

Статистикой критерия является:

$$
t = \frac{\bar{X} - \mu_0}{s / \sqrt{n}}
$$

При верности нулевой гипотезы $H_0$ статистика критерия имеет распределение Стьюдента с $n - 1$ степенями свободы:

$$
t \sim T_{n-1}
$$

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) равно:

Для двусторонней альтернативы:
$$
p = 2 \times Probability(T_{n-1} \ge |t|)
$$

Для односторонней альтернативы (больше):
$$
p = Probability(T_{n-1} \ge t)
$$

Для односторонней альтернативы (меньше):
$$
p = Probability(T_{n-1} \le t)
$$

**Тренировочная задача №2**

Возьмём условие **тренировочной задачи №1**, но теперь для сравнения используем **Одновыборочный Т-тест**. **Одновыборочный Т-тест** может быть применён так как $n = 1000$ (достаточно велико для применения ЦПТ).

**Условие:** Историческая конверсия в целевое действие у сегмента равна $p_0 = \mu_0 = 0.7$%. Мы отправляем выборке из 1000 клиентов СМС с призывом совершить целевое действие. После эксперимента мы выяснили, что в экспериментальной группе в целевое действие сконвертировалось 14 клиентов.

**Задача:** сравнить конверисю в экспериментальной группе ($p$, которое равно $\mu$) с исторической конверсией по сегменту ($p_0$, которое равно $\mu_0$) при помощи **одновыборочного Т-теста** и сделать вывод о том значимы ли различия на уровне значимости $alpha = 0.05$

Проверяемые гипотезы (двусторонняя гипотеза, $\mu$ не равна $\mu_0$):
$$
H_0: \mu = 0.007 \\
H_1: \mu <> 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $\mu$ больше $\mu_0$):
$$
H_0: \mu \le 0.007 \\
H_1: \mu > 0.007 \\
$$

Проверяемые гипотезы (односторонняя гипотеза, $\mu$ меньше $\mu_0$):
$$
H_0: \mu \ge 0.007 \\
H_1: \mu < 0.007 \\
$$

In [6]:
from ab_utils.t_test import onesample_p_value, onesample_p_value_manual

In [7]:
n = 1000          # количество пользователей (испытаний)
k = 14            # число конверсий (успехов)
mu0 = 0.007        # ожидаемая конверсия 0.7% = 0.007
alpha = 0.05      # уровень значимости alpha

mu = k / n
print(f"Наблюдаемая конверсия: {mu:.1%}")
print(f"Ожидаемая конверсия (H0): {mu0:.1%}")

# Создаём массив, в котором 14 клиентов из 1000 конвертировалось в целевое действие
x_arr = np.zeros(n)
x_arr[:k] = 1

Наблюдаемая конверсия: 1.4%
Ожидаемая конверсия (H0): 0.7%


In [8]:
alt_to_h_zero = {
    'two-sided': f'p {round(p * 100.0, 1)}% равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% больше либо равна p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% меньше либо равна p_0 {round(p0 * 100.0, 1)}%'
}

alt_to_h_one = {
    'two-sided': f'p {round(p * 100.0, 1)}% НЕ равна p_0 {round(p0 * 100.0, 1)}%',
    'less': f'p {round(p * 100.0, 1)}% меньше p_0 {round(p0 * 100.0, 1)}%',
    'greater': f'p {round(p * 100.0, 1)}% больше p_0 {round(p0 * 100.0, 1)}%'
}

for alt in ['two-sided', 'less', 'greater']:
    p_scipy = onesample_p_value(x_arr, mu0, alternative = alt)[0]
    p_manual = onesample_p_value_manual(x_arr, mu0, alternative = alt)[0]

    print(f"\nАльтернатива: {alt}")
    print(f"p-value (scipy):  {p_scipy:.6f}; Гипотеза H_0 ({alt_to_h_zero[alt]}) отвергается на уровне {alpha} в пользу H_1 ({alt_to_h_one[alt]}): {'да' if p_scipy < alpha else 'нет'}")
    print(f"p-value (manual): {p_manual:.6f}")


Альтернатива: two-sided
p-value (scipy):  0.059974; Гипотеза H_0 (p 1.4% равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% НЕ равна p_0 0.7%): нет
p-value (manual): 0.059974

Альтернатива: less
p-value (scipy):  0.970013; Гипотеза H_0 (p 1.4% больше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% меньше p_0 0.7%): нет
p-value (manual): 0.970013

Альтернатива: greater
p-value (scipy):  0.029987; Гипотеза H_0 (p 1.4% меньше либо равна p_0 0.7%) отвергается на уровне 0.05 в пользу H_1 (p 1.4% больше p_0 0.7%): да
p-value (manual): 0.029987


**Обратите внимание** на результат **Одновыборочного Т-теста** при тестировании двусторонней гипотезы - результат отличается от результата **Биномиального теста**. При таких маленьких конверсиях, **Одновыборочный Т-тест** не подходит, так как не выполняется требование о нормальном распределении среднего. Как проверять корректность выбранного критерия будет рассказано в секции "3. Проверка корректности выбора критерия"

### 1.2.1. Сравнение больших конверсий между группами. **Z-тест для пропорций (Z-test)**

#### Проверяемые гипотезы (двусторонняя гипотеза)
$$
H_0: p_1 = p_2 \\
H_1: p_1 <> p_2
$$
где $p_1$ - значение конверсии в экспериментальной группе, $p_2$ - значение конверсии в контрольной группе

#### Условия применимости критерия
1. Испытания независимы
2. $n_1 \times \hat{p_1} \ge 50$, $n_1 \times (1 - \hat{p_1}) \ge 50$, $n_2 \times \hat{p_2} \ge 50$, $n_2 \times (1 - \hat{p_1}) \ge 50$. В противном случае лучше применять "Точный тест Фишера"

#### Статистика критерия
Пусть
- $x_1$, $x_2$ - количество успехов,
- $n_1$, $n_2$ - размеры выборок,
- $\hat{p_1} = \frac{x_1}{n_1}$, $\hat{p_2} = \frac{x_2}{n_2}$,
- объединённая пропорция $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$.

Статистикой критерия является:

$$
z = \frac{\hat{p_1} - \hat{p_2}}{\sqrt{\hat{p} \times (1 - \hat{p}) \times (\frac{1}{n_1} + \frac{1}{n_2})}}
$$

При верности нулевой гипотезы $H_0$ статистика критерия имеет нормальное распределение со средним равным $0$ и стандартным отклонением равным $1$:

$$
z \sim N(0, 1)
$$

#### Расчёт p-value
$p-value$ (вероятность получить такое же или более экстремальное значение статистики) равно:

Для двусторонней альтернативы:
$$
p = 2 \times Probability(N(0, 1) \ge |z|)
$$

Для односторонней альтернативы (больше):
$$
p = Probability(N(0, 1) \ge z)
$$

Для односторонней альтернативы (меньше):
$$
p = Probability(N(0, 1) \le z)
$$

## 2. Расчёта размера выборки (Sample Size) и минимального детектируемого эффекта (MDE)

...

## 3. Проверка корректности выбора критерия на синтетических данных

...